## Blackjack

It has all functionalities necessary for multiple plyers to play multiple rounds of blackjack but needs some more improvement to the code for a better playing experience

In [ ]:
import random
from IPython.display import clear_output

# Default Player Names and all variables for card values
suits = ('Hearts', 'Diamonds', 'Spades', 'Clubs')
ranks = ('Two', 'Three', 'Four', 'Five', 'Six', 'Seven', 'Eight', 'Nine', 'Ten', 'Jack', 'Queen', 'King', 'Ace')
values = {'Two':2, 'Three':3, 'Four':4, 'Five':5, 'Six':6, 'Seven':7, 'Eight':8,'Nine':9, 'Ten':10, 'Jack':10, 'Queen':10,
          'King':10, 'Ace':11}
default_player_name = ['Player 1', 'Player 2', 'Player 3', 'Player 4', 'Player 5', 'Player 6', 'Player 7', 'Player 8']
players_list = []
dealer = ''

# A single card
class Card:
    def __init__(self, suit, rank):
        self.suit = suit
        self.rank = rank
        self.value = values[rank]
    
    def __str__(self):
        return '{} of {}'.format(self.rank, self.suit)

# A deck of cards that can be shuffled and the top card can be dealt
class Deck:
    def __init__(self):
        self.cards = []
        for suit in suits:
            for rank in ranks:
                self.cards.append(Card(suit, rank))

    def __str__(self):
        deck_str = ''
        for card in self.cards:
            deck_str += card.__str__() + '\n'
        return deck_str
    
    def __len__(self):
        return len(self.cards)
    
    def shuffle(self):
        random.shuffle(self.cards)
        
    def deal_one_card(self):
        return self.cards.pop()

# Parent for dealer & players
# isn't initialized anywhere, just has show_hand, add_card & calc_hand_value methods to avoid code duplication
class GameParticipant:
    def show_hand(self):
        hand_str = ''
        for card in self.hand:
            hand_str += card.__str__() + '\n'
        return hand_str
    
    def add_card(self, card):
        self.hand.append(card)
        if card.rank == 'Ace':
            self.soft_ace += 1
        self.calc_hand_value(card)
    
    def calc_hand_value(self, card):
        self.hand_value += values[card.rank]
    
        while self.hand_value > 21 and self.soft_ace:
            self.hand_value -= 10
            self.soft_ace -= 1

# Dealer - holds a stack of decks and deals cards to players
class Dealer(GameParticipant):  
    def __init__(self, no_of_decks):
        self.decks = []
        self.hand = []
        self.no_of_decks = no_of_decks
        self.soft_ace = 0
        self.hand_value = 0
        self.set_decks()
    
    def __str__(self):
        return 'Dealer with {} deck(s)'.format(len(self.decks))
    
    def set_decks(self):
        for i in range(self.no_of_decks):
            self.decks.append(Deck())
        self.shuffle_deck_stack()
    
    def shuffle_deck_stack(self):
        random.shuffle(self.decks)
        for d in self.decks:
            d.shuffle()
    
    def deal(self):
        try:
            return self.decks[0].deal_one_card()
        except IndexError:
            try:
                self.decks.pop(0)
                return self.decks[0].deal_one_card()
            except:
                print('Deck Over...shuffling!')
                self.set_decks()
                return self.decks[0].deal_one_card()

# Player - one of the players that can play the game, holds card in hand during the round and has a chip stack to which
# chips can be added or removed from depending on the result of the hand
class Player(GameParticipant):
    def __init__(self, name, position=0, chips=100):
        self.name = name
        self.position = position
        self.chips = chips
        self.hand = []
        self.soft_ace = 0
        self.hand_value = 0
        self.bet = 0
        self.playing = True
    
    def __str__(self):
        print('{} is sitting at {} with {} chips'.format(self.name, self.position, self.chips))
    
    def hand_result(self, win_flag):
        self.chips += win_flag*self.bet
        self.bet = 0
        self.hand = []
        self.hand_value = 0


In [ ]:
# Set Player Names & Starting Chip Stacks
def player_initializer(players, no_of_players):
    for i in range(no_of_players):
        pl_name = input('Enter {} name: '.format(default_player_name[i])).capitalize()
        if pl_name == '' or pl_name.isspace():
            pl_name = default_player_name[i]
        
        while True:
            stack_temp = input('Enter the number of starting chips for {} (10 - 1000): '.format(pl_name))
            if not stack_temp.isdigit():
                print('Chips can only be whole numbers!')
                continue
            elif int(stack_temp) < 10:
                print('Minimum starting chips should be 10')
                continue
            elif int(stack_temp) > 1000:
                print('Maximum starting chips can be 1000')
                continue
            else:
                pl_stack = int(stack_temp)
                break
    
        players.append(Player(pl_name, 0, pl_stack))

# Deals first 2 cards of the round to all particiants
def deal_initial_cards(pl_list, dlr):
    for i in range(2):
        for pl in pl_list:
            pl.add_card(dlr.deal())

        dlr.add_card(dlr.deal())

# Hit - Check if this can be removed in next refactor 
def hit(pl, dlr):
    pl.add_card(dlr.deal())

# Play Ball!
def play_game(first = True):
    global dealer, players_list

# Quick and Dirty variable initialization for testing
    testing = False
    if testing and first:
        game_decks = 1
        no_of_players = 1
        first = False
        players_list = []
        dealer = ''
        players_list.append(Player('A', 0, 100))
        dealer = Dealer(int(game_decks))
        dealer.shuffle_deck_stack()
        print(dealer)
        
    while first:
        game_decks = 0
        no_of_players = 0
        players_list = []
        dealer = ''
#         continuous_shuffle = ''

#         shuffle_type = input('Do you want Continuous Shuffle (Y/N):')
#         Really simple to implement, the dealer will be given a new deck every hand...if this is true, we can play with
#         one deck only. The current implementation runs till the end of the deck stack and gives the dealer a new stack
#         if cards run out even in the middle of the game. Need to implement a deck break at 50-75% of all cards and
#         shuffle the deck if the total cards in stack is lower than (1 - break point) * total cards in stack 
        while True:
            game_decks = input("Please enter the number of decks you'd like to play with (1 - 8): ")

            if not game_decks.isdigit() or int(game_decks) not in range(1, 9):
                print('Please enter an integer between 1 and 8')
                continue
            else:
                print('No of decks: {}'.format(game_decks))
                break
    
        while True:
            no_of_players = input('Please enter the number of players (1 - 8): ')

            if not no_of_players.isdigit() or int(no_of_players) not in range(1, 9):
                print('Please enter an integer between 1 and 8')
                continue
            else:
                print('No of players: {}'.format(no_of_players))
                break
        
        player_initializer(players_list, int(no_of_players))

        dealer = Dealer(int(game_decks))
        dealer.shuffle_deck_stack()
        print(dealer)
        
        first = False
    
    dealer.hand = []
    dealer.hand_value = 0
#     Check if all players have busted
#     all_bets = 0
    
    for pl in players_list:
        if testing:
            pl.bet = 1
            break
        
        clear_output()
        print('Place your bets!')
        
        while True:
            bet_amt = input('Enter bet amount for {}: '.format(pl.name))
            if not bet_amt.isdigit():
                print('Please enter a whole number.')
                continue
            elif int(bet_amt) > pl.chips:
                print("Bet can't be higher than your current chip count.")
                continue
            else:
                pl.bet = int(bet_amt)
                break
    
    deal_initial_cards(players_list, dealer)
      
    for pl in players_list:
        if testing:
            for i in range(40):
                hit(pl, dealer)
            print("{}'s hand value is {}\n".format(pl.name, pl.hand_value))
            break
        
        clear_output()
        print("Dealer's face up card is '{}' and current hand value is {}\n".format(dealer.hand[1], values[dealer.hand[1].rank]))
        print("{}'s hand is:\n{}and hand value is {}".format(pl.name, pl.show_hand(), pl.hand_value))
        
        while True:
            hit_or_stand = input("Would you like to hit or stand? (Enter 'h' or 's'): ").lower()
            
            if hit_or_stand == 'h':
                clear_output()
                print('{} hits!\n'.format(pl.name))
                hit(pl, dealer)
                print("{}'s hand is:\n{}and hand value is {}\n".format(pl.name, pl.show_hand(), pl.hand_value))
            
                if pl.hand_value > 21:
                    print('{} busts :('.format(pl.name))
                    pl.hand_result(-1)
                    input('\nPress any key to continue')
                    break
                
            elif hit_or_stand == 's':
                clear_output()
                print('{} stands!\n'.format(pl.name))
                print("{}'s final hand is:\n{}and hand value is {}".format(pl.name, pl.show_hand(), pl.hand_value))
                input('\nPress any key to continue')
                break
            else:
                print('Please enter h or s only!')

#     If all busted, stop current round and check if players want to play on
#     for pl in players_list:
#         all_bets = pl.hand_value
    
#     if all_bets == 0:
#         clear_output()
#         print('All players busted!\nStarting next round.')
#         input('\nPress any key to continue')
#         play_game(False)
    
    if not testing:
        clear_output()
    print("\nDealer's Turn!")
    print('Dealer stands on 17!\n')
    print("Dealer's hand is:\n{}and hand value is {}".format(dealer.show_hand(), dealer.hand_value))
    input('\nPress any key to continue')
    
    while dealer.hand_value < 17:
        if testing:
            break
        
        clear_output()
        print('Dealer hits!\n')
        hit(dealer, dealer)
        print("Dealer's hand is:\n{}and hand value is {}".format(dealer.show_hand(), dealer.hand_value))
        input('\nPress any key to continue')
    else:
        if dealer.hand_value < 22:
            clear_output()
            print('Dealer stands!\n')
            print("Dealer's final hand is:\n{}and hand value is {}".format(dealer.show_hand(), dealer.hand_value))
            input('\nPress any key to continue\n')
        else:
            clear_output()
            print('Dealer busts!\n')
            print("Dealer's final hand is:\n{}and hand value is {}".format(dealer.show_hand(), dealer.hand_value))
            input('\nPress any key to continue\n')
    
    if dealer.hand_value > 21:
        print("\nDealer busts! All player's with active hands win!")
        for pl in players_list:
            pl.hand_result(1)
            print("{}'s chip count is {}".format(pl.name, pl.chips))
    else:
        for pl in players_list:
            if pl.bet == 0:
                print("{}'s final hand is:\n{}and hand value is {}".format(pl.name, pl.show_hand(), pl.hand_value))
                print('{} already busted :('.format(pl.name))
                print("{}'s chip count is {}".format(pl.name, pl.chips))
                continue

            if pl.hand_value > dealer.hand_value:
                print("{}'s final hand is:\n{}and hand value is {}".format(pl.name, pl.show_hand(), pl.hand_value))
                print('\n{} WINS!!!'.format(pl.name))
                pl.hand_result(1)
            elif pl.hand_value < dealer.hand_value:
                print("{}'s final hand is:\n{}and hand value is {}".format(pl.name, pl.show_hand(), pl.hand_value))
                print('\n{} loses :('.format(pl.name))
                pl.hand_result(-1)
            else:
                print("{}'s final hand is:\n{}and hand value is {}".format(pl.name, pl.show_hand(), pl.hand_value))
                print('Push!')
                pl.hand_result(0)
            
            print("{}'s chip count is {}\n".format(pl.name, pl.chips))
    
    for pl in players_list:
        if pl.chips == 0:
            print("{}'s chips are over and they leave the table!".format(pl.name))
            pl.playing = False
        else:
            play_on = input('Would {} like to play another round? Enter "Y" to play on, any other key to exit.'.format(pl.name)).lower()
            if play_on != 'y':
                print('{} leaves the table with {} chips!'.format(pl.name, pl.chips))
                pl.playing = False
    
    players_list = [pl for pl in players_list if pl.playing == True]
#     print(len(dealer.decks), len(dealer.decks[0]))
    
    if len(players_list) > 0:
        play_game(False)
    else:
        print("No more players to continue. Thanks for playing!")

In [ ]:
play_game()